# Asset Class Trend Following 策略回測 (2025)
本筆記本重現 Asset Class Trend Following 策略在 2025 年的表現。

In [ ]:
import pandas as pd
import numpy as np
import xlsxwriter

SMA_PERIOD = 69
ROC_PERIOD = 23
STOP_LOSS_PCT = 0.09
INITIAL_CAPITAL = 30000000
DATA_FILE = '個股2.xlsx'

In [ ]:
def clean_data(filepath):
    df_raw = pd.read_excel(filepath, header=None)
    stock_codes = df_raw.iloc[0, 2:].astype(str).values
    stock_names = df_raw.iloc[1, 2:].values
    dates = pd.to_datetime(df_raw.iloc[2:, 1])
    prices = df_raw.iloc[2:, 2:].astype(float)
    prices.index = dates
    prices.columns = stock_codes
    prices = prices.bfill().ffill()
    prices = prices.dropna(axis=1, how='all')
    return prices, dict(zip(stock_codes, stock_names))

class Backtester:
    def __init__(self, prices, initial_capital=30000000):
        self.prices = prices.values
        self.dates = prices.index
        self.assets = prices.columns
        self.initial_capital = initial_capital

    def run(self, sma_period, roc_period, stop_loss_pct):
        prices_df = pd.DataFrame(self.prices, index=self.dates, columns=self.assets)
        sma = prices_df.rolling(window=sma_period).mean().values
        roc = prices_df.pct_change(periods=roc_period).values
        
        capital = self.initial_capital
        portfolio = {} # asset_idx -> info
        equity_curve = np.full(len(self.dates), self.initial_capital, dtype=float)
        rebalance_log = []
        
        start_idx = 68 # 2025-01-10 (Fri) - Rebalance signals start here
        
        for i in range(start_idx, len(self.dates)):
            # 1. Daily Valuation and Peak Update
            current_val = capital
            for asset_idx, info in portfolio.items():
                p = self.prices[i][asset_idx]
                if p > info['max_price']: info['max_price'] = p
                current_val += info['shares'] * p
            equity_curve[i] = current_val
            
            # 2. Rebalance (Signal T=i-1, Execution T+1=i)
            # Rebalance cycle is 5 trading days
            if i > start_idx and (i - 1 - start_idx) % 5 == 0:
                signal_idx = i - 1
                exec_idx = i
                exec_date = self.dates[exec_idx]
                
                # Signal Generation (T = signal_idx)
                elig = (self.prices[signal_idx] > sma[signal_idx]) & (roc[signal_idx] > 0)
                elig_idxs = np.where(elig)[0]
                elig_rocs = roc[signal_idx][elig_idxs]
                top_k = min(3, len(elig_idxs))
                target_idxs = elig_idxs[np.argsort(elig_rocs)[-top_k:][::-1]]
                target_set = set(target_idxs)
                
                current_assets = set(portfolio.keys())
                
                # Sells (Not in Top 3)
                to_sell = current_assets - target_set
                for idx in to_sell:
                    info = portfolio.pop(idx)
                    sell_p = self.prices[exec_idx][idx]
                    capital += info['shares'] * sell_p
                    hit_sl = self.prices[signal_idx][idx] < info['max_price'] * (1 - stop_loss_pct)
                    rebalance_log.append({
                        '日期': exec_date,
                        '股票代號': self.assets[idx],
                        '狀態': '賣出',
                        '價格': sell_p,
                        '原因': '停損' if hit_sl else '再平衡',
                        '動能值': roc[signal_idx][idx]
                    })
                
                # Keeps (In Top 3)
                to_keep = current_assets & target_set
                for idx in to_keep:
                    rebalance_log.append({
                        '日期': exec_date,
                        '股票代號': self.assets[idx],
                        '狀態': '保持',
                        '價格': self.prices[exec_idx][idx],
                        '原因': '趨勢持續',
                        '動能值': roc[signal_idx][idx]
                    })
                
                # Buys (New in Top 3)
                to_buy = target_set - current_assets
                slot_cap = self.initial_capital / 3
                for idx in to_buy:
                    buy_p = self.prices[exec_idx][idx]
                    shares = slot_cap // buy_p
                    if shares > 0 and capital >= shares * buy_p:
                        capital -= shares * buy_p
                        portfolio[idx] = {'shares': shares, 'max_price': buy_p, 'buy_price': buy_p}
                        rebalance_log.append({
                            '日期': exec_date,
                            '股票代號': self.assets[idx],
                            '狀態': '買進',
                            '價格': buy_p,
                            '原因': '符合趨勢',
                            '動能值': roc[signal_idx][idx]
                        })
                
                # Re-calculate equity for execution day after trades
                new_val = capital
                for idx, info in portfolio.items():
                    new_val += info['shares'] * self.prices[exec_idx][idx]
                equity_curve[exec_idx] = new_val

        return pd.Series(equity_curve, index=self.dates), pd.DataFrame(rebalance_log)

def calculate_metrics(equity_curve, rebalance_log, start_date='2025-01-01', end_date='2025-12-31'):
    eq = equity_curve[start_date:end_date]
    if eq.empty: return {}
    base_val = 30000000.0
    total_ret = (eq.iloc[-1] / base_val) - 1
    days = (eq.index[-1] - eq.index[0]).days
    cagr = (1 + total_ret) ** (365.25 / days) - 1 if days > 0 else 0
    rolling_max = eq.cummax()
    max_dd = ((eq - rolling_max) / rolling_max).min()
    calmar = cagr / abs(max_dd) if max_dd != 0 else 0
    
    # Win Rate calculation from log
    positions = {}
    wins = []
    for _, row in rebalance_log.iterrows():
        ticker = row['股票代號']
        if row['日期'] < pd.to_datetime(start_date): continue
        if row['狀態'] == '買進': positions[ticker] = row['價格']
        elif row['狀態'] == '賣出' and ticker in positions:
            wins.append(row['價格'] > positions[ticker])
            del positions[ticker]
    win_rate = np.mean(wins) if wins else 0
    
    return {'CAGR': cagr, 'MaxDD': max_dd, 'Calmar': calmar, 'WinRate': win_rate}


In [ ]:
prices, code_to_name = clean_data(DATA_FILE)
bt = Backtester(prices, INITIAL_CAPITAL)
eq, log = bt.run(SMA_PERIOD, ROC_PERIOD, STOP_LOSS_PCT)

def calculate_metrics_2025(equity_curve, rebalance_log):
    start_date, end_date = '2025-01-01', '2025-12-31'
    eq = equity_curve[start_date:end_date]
    base_val = 30000000.0
    total_ret = (eq.iloc[-1] / base_val) - 1
    days = (eq.index[-1] - eq.index[0]).days
    cagr = (1 + total_ret) ** (365.25 / days) - 1
    rolling_max = eq.cummax()
    max_dd = ((eq - rolling_max) / rolling_max).min()
    calmar = cagr / abs(max_dd) if max_dd != 0 else 0
    return {'CAGR': cagr, 'MaxDD': max_dd, 'Calmar': calmar}

metrics = calculate_metrics_2025(eq, log)
print('2025 績效指標：', metrics)

In [ ]:
plateau = []
for sma in [65, 67, 69, 71, 73]:
    eq_p, _ = bt.run(sma, ROC_PERIOD, STOP_LOSS_PCT)
    m = calculate_metrics_2025(eq_p, _)
    plateau.append({'SMA': sma, 'CAGR': f'{m[\'CAGR\']:.2%}', 'MaxDD': f'{m[\'MaxDD\']:.2%}', 'Calmar': f'{m[\'Calmar\']:.2f}'})
pd.DataFrame(plateau)